In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Intro") \
    .getOrCreate()

In [4]:
dat = spark.read.options(samplingRatio=0.005).json('tweet-data/')

In [5]:
from itertools import combinations

pairs = dat.rdd \
    .map(lambda r: r.entities) \
    .filter(lambda e: e and len(e.hashtags) > 1) \
    .map(lambda e: e.hashtags) \
    .map(lambda tags: {t.text for t in tags}) \
    .flatMap(lambda li: combinations(li, 2)) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda a,b: a + b) \
    .filter(lambda t: t[1] > 2)

In [6]:
pairs.cache()

lookup = pairs.flatMap(lambda t: t[0]).distinct().zipWithIndex().collectAsMap()

N = len(lookup)

In [ ]:
popular_tags

In [8]:
N

13556

In [8]:
def get_idx(tags, lookup):
    return [lookup[t] for t in tags]

def local_reducer(m,t):
    i,j = t[0]
    m[i,j] = t[1]
    return m

In [30]:
from scipy.sparse import dok_matrix
from operator import add
import numpy as np 

m = dok_matrix((N, N), dtype=np.int32)

out = pairs \
    .map(lambda t: (get_idx(t[0], lookup), t[1])) \
    .aggregate(m, local_reducer, add)

In [31]:
out = out.asformat('csr')
out += out.T

In [32]:
out[lookup['NewGreenDeal'], lookup['Brexit']]

13

In [33]:
out[lookup['Memes'], lookup['Merkel']]

25

In [34]:
out.sum()

1174992